# Joining with Pandas

Joining allows you to combine `Series` and `DataFrame`s.

## Import Pandas

As usual, let's import Pandas

In [1]:
import pandas as pd

# Concatenate and Append

## Concatenate

`concat` sticks dataframes together, either on top of each other, or next to each other.

Let's take a look at the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) for `concat`, which says it will "Concatenate pandas objects along a particular axis with optional set logic along the other axes.":

```python
pandas.concat(objs, axis=0, join='outer', join_axes=None, 
              ignore_index=False, keys=None, levels=None, names=None, 
              verify_integrity=False, sort=None, copy=True)
```

Let's see it in action with two dataframes, `df1` and `df2`.

In [2]:
df1 = pd.DataFrame( { 'letter': ['a', 'b'], 'number': [1,2]} )
df1

,letter,number
0,a,1
1,b,2


In [4]:
df2 = pd.DataFrame([ ['c', 3], ['d', 4], ['e', 5] ], columns=['letter', 'number'])
df2

,letter,number
0,c,3
1,d,4
2,e,5


Next, let's stick the dataframes on top of each other using `concat`. 

In [7]:
pd.concat([df1, df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4
2,e,5


Finally, let's stick the dataframes **next** to each other using `concat`. We want it to concatenate along the columns of the table, so the `axis='columns'` kwarg will help us here:

In [8]:
pd.concat([df1, df2], axis='columns')

,letter,number,letter,number
0,a,1.0,c,3
1,b,2.0,d,4
2,NaN,NaN,e,5


## Append

`append` is very similar to `concat`, except it limits itself to a specific case of `concat` -- specifically where `axis=0` (stack on top of each other) and `join=outer` (how to handle the axis of the second dataframe).

Let's take a look at the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html), which says `append` will "Append rows of other to the end of caller, returning a new object":

```python
DataFrame.append(self, other, ignore_index=False, verify_integrity=False, sort=None)
```

In [9]:
df3 = df1.append(df2)
df3

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4
2,e,5


For almost all cases, `concat` can do all that `append` can do (and more), so we'll focus on `concat`.

Also note that `append` is a `DataFrame` and `Series` method, whereas `concat` is a general pandas library function.

Notice that in both cases of `concat` and `append`, the index was also simply stacked together. You can regenerate the index with `reset_index`, if you wish:

In [13]:
df3.reset_index(inplace=True, drop=True)
df3

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5


# Joining

## Join

`join` allows us to combine two dataframes by using a matching column known as a `key`.

With `join`, the `key` joining the table is always the `index` of the first table with (by default) the index of the second table. 

Let's take a look at the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html), which says `join` will "Join columns with other DataFrame either on index or on a key column":

```python
DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
```

Let's take it out for a spin. First, create two dataframes.

In [14]:
df1 = pd.DataFrame([ ['a', 1], ['b', 2], ['c', 3], ['d', 4] ], columns=['letter', 'number'])
df1

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [15]:
df2 = pd.DataFrame([ ['e', 5], ['f', 6] ], columns=['letter', 'number'])
df2

,letter,number
0,e,5
1,f,6


Now, lets `join` these two dataframes. Note that we will 'line up', the two dataframes based on their indicies.

When joining dataframes where column names are not unique, we will need to supply a `lsuffix` or `rsuffix` kwarg.

This is appended to the end of the column name of the returned, joined dataframe to differentiate and identify the source column. Here, we'll use `_df1` to identify that the column shown came from the `df1` dataframe, and `_df2` as a suffix to identify its origin as the `df2` dataframe. 

In [19]:
df1.join(df2, lsuffix='_df1', rsuffix='_df2')

,letter_df1,number_df1,letter_df2,number_df2
0,a,1,e,5.0
1,b,2,f,6.0
2,c,3,NaN,NaN
3,d,4,NaN,NaN


Note how we have joined the two dataframes on their indicies, which creates a null for rows of index 2 and 3 in the `_df2` values. This is expected and correct, as those indicies don't exist in `df2`.

The default behavior of `join` is `left`. That means we keep everything from the left table, and disregard anything on the right table that doesn't have a matching `key`.

If `df2` had more elements than `df1`:

In [20]:
df2 = pd.DataFrame([['e', 5], ['f', 6], ['g', 7], ['h', 8], ['Z', 9000]], columns=['letter', 'number'])
df2

,letter,number
0,e,5
1,f,6
2,g,7
3,h,8
4,Z,9000


And we join them together again,  you'll see that the `Z: 9000` data point will not be present in the joined table. As mentioned before, a `left` join keeps everything from the left table and disregards anything on the right table that doesn't have a matching `key`.

In [22]:
df1.join(df2, lsuffix='_df1', rsuffix='_df2', how='left')

,letter_df1,number_df1,letter_df2,number_df2
0,a,1,e,5
1,b,2,f,6
2,c,3,g,7
3,d,4,h,8


In [23]:
df1.join(df2, lsuffix='_df1', rsuffix='_df2', how='right')

,letter_df1,number_df1,letter_df2,number_df2
0,a,1.0,e,5
1,b,2.0,f,6
2,c,3.0,g,7
3,d,4.0,h,8
4,NaN,NaN,Z,9000


### Join Types

We can change the join type with the `how` kwarg.

For reference, here are the common types of joins. 

The type of join (**left**) we just performed above is the upper-left most figure. We'll briefly look at **inner** and **right** joins but other join types won't be covered in detail in this course unfortunately.

<img src="assets/join_types.jpg" width="500" />

## Merge

Similar to `join` is `merge`. The difference between the two is the *keying behavior*. `merge` has a richer API (more functionality) and allows one to join on columns in the source dataframe *other than the index*.

Because `merge` can effectively do everything that `join` can do, and more - at this beginner level it is recommended to use `merge` unless code brevity is the top concern. 

Let's take a look at the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html), which says `merge` will "Merge DataFrame or named Series objects with a database-style join":

```python
pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, 
             left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), 
             copy=True, indicator=False, validate=None)
```

Note that `merge` comes in *both* a [`DataFrame` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) flavor as well as a [general function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) flavor.

Below, we'll be using the general function flavor:

In [29]:
pd.merge(df1, df2, how='left', left_index=True, right_index=True, suffixes=('_df1', '_df2') )

,letter_df1,number_df1,letter_df2,number_df2
0,a,1,e,5
1,b,2,f,6
2,c,3,g,7
3,d,4,h,8


Note that we've achieved the same exact output as we did with `join`, but it took a little more explicit work. Let's run through the arguments for clarity:

* `df1`: this is the first dataframe, and considered to be on the **left**
* `df2`: this is the second dataframe, considered to be on the **right**
* `how='left'`: this states the type of join; see the above diagram
* `left_index=True`: this uses the index of `df1` as the join key for the left table
* `right_index=True`: this uses the index of `df2` as the join key for the right table
* `suffixes`: this places `_df?` after column names which came from `df?`

How does the situation change if we use `how='right'`?

In [30]:
pd.merge(df1, df2, how='right', left_index=True, right_index=True, suffixes=('_df1', '_df2') )

,letter_df1,number_df1,letter_df2,number_df2
0,a,1.0,e,5
1,b,2.0,f,6
2,c,3.0,g,7
3,d,4.0,h,8
4,NaN,NaN,Z,9000


### Merge on Non-Index Columns

This brings us to our next point: merging on columns that are not the index columns. This is very common in SQL joins and this technique can be used to join just about any DataFrame.

First, let's create some more realistic data - stocks!

In [32]:
stock_name = pd.DataFrame({
    'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'],
    'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']
})

opening_price = pd.DataFrame({
    'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'],
    'OpeningPrice': [217.51, 96.54, 51.45, 1703.34]
})

yearly_high = pd.DataFrame({
    'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'],
    '52wkHigh': [60.79, 2050.49, 233.47, 110.11]
})

print(stock_name, opening_price, yearly_high, sep='\n\n')

  Symbol            Name
0   AMZN          Amazon
1    DHR         Danaher
2    DAL  Delta Airlines
3   AAPL           Apple

  Symbol  OpeningPrice
0   AAPL        217.51
1    DHR         96.54
2    DAL         51.45
3   AMZN       1703.34

  Symbol  52wkHigh
0    DAL     60.79
1   AMZN   2050.49
2   AAPL    233.47
3    DHR    110.11


Now, let's join the `opening_price` and `yearly_high` dataframes together:

In [35]:
pd.merge(opening_price, yearly_high, on='Symbol')

,Symbol,OpeningPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


Notice that the dataframes are not in any particular order. This is intentional. Because of this, notice that the dataframe on the left (`opening_price`) dictates the order of the dataframe on the right, `yearly_high`.

In this particular case, we could also have omitted the `left_on` and `right_on` kwargs, as `Symbol` is the only common column between the two dataframes.

Note that the shared key between the two dataframes is exempt from having a `suffix` applied to it. 

### Now it's your turn!

* `merge` the `stock_name` and `opening_price` dataframes and inspect the result:

In [36]:
pd.merge(stock_name, opening_price, on='Symbol')

,Symbol,Name,OpeningPrice
0,AMZN,Amazon,1703.34
1,DHR,Danaher,96.54
2,DAL,Delta Airlines,51.45
3,AAPL,Apple,217.51


* `merge` all three dataframes together and inspect the result (**Hint**: Remember that `merge` comes in two flavors!):

In [40]:
pd.merge(stock_name, opening_price, on='Symbol') \
  .merge(yearly_high, on='Symbol')

,Symbol,Name,OpeningPrice,52wkHigh
0,AMZN,Amazon,1703.34,2050.49
1,DHR,Danaher,96.54,110.11
2,DAL,Delta Airlines,51.45,60.79
3,AAPL,Apple,217.51,233.47


# Large Example: Adventure Works

<img src="assets/adventure_works_logo.png" align="right" />

Here are data dictionaries for the data we'll be using:

* [Products](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html)
* [Product Subcategories](https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubCategory.html)
* [Orders](https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html)
* [Line Items](https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html)

In [42]:
products              = pd.read_csv('data/product.csv')
product_subcategories = pd.read_csv('data/product_subcategories.csv')
orders                = pd.read_csv('data/orders.csv', nrows=100)
line_items            = pd.read_csv('data/line_items.csv')

## Join Product Tables

It would appear that every product should have a `ProductSubCategory`, which makes sense: Each product is a type of a product (or what they chose to call `ProductSubCategory`.

In [44]:
print(products.shape)
products.head(3)

(504, 24)


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,0,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,...,1,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000


In [47]:
print(product_subcategories.shape)
product_subcategories

(37, 4)


,ProductSubcategoryID,ProductCategoryID,Name,ModifiedDate
0,1,1,Mountain Bikes,2008-04-30 00:00:00
1,2,1,Road Bikes,2008-04-30 00:00:00
2,3,1,Touring Bikes,2008-04-30 00:00:00
3,4,2,Handlebars,2008-04-30 00:00:00
4,5,2,Bottom Brackets,2008-04-30 00:00:00
5,6,2,Brakes,2008-04-30 00:00:00
6,7,2,Chains,2008-04-30 00:00:00
7,8,2,Cranksets,2008-04-30 00:00:00
8,9,2,Derailleurs,2008-04-30 00:00:00
9,10,2,Forks,2008-04-30 00:00:00


Let's do an "Inner Join" (the default) on the `products` and `product_subcategories` tables, with the `ProductSubcategoryID` key:

In [52]:
pd.set_option('display.max_columns', 30)

merged_products = pd.merge(products, product_subcategories, on='ProductSubcategoryID', suffixes=('_p', '_ps'))
print(merged_products.shape)
merged_products.head()

(295, 27)


,ProductID,Name_p,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ModifiedDate_p,ProductCategoryID,Name_ps,ModifiedDate_ps
0,680,"HL Road Frame - Black, 58",FR-R92B-58,1,1,Black,500,375,1059.3100,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000,2,Road Frames,2008-04-30 00:00:00
1,706,"HL Road Frame - Red, 58",FR-R92R-58,1,1,Red,500,375,1059.3100,1431.5,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000,2,Road Frames,2008-04-30 00:00:00
2,717,"HL Road Frame - Red, 62",FR-R92R-62,1,1,Red,500,375,868.6342,1431.5,62,CM,LB,2.30,1,R,H,U,14.0,6.0,2011-05-31 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000,2,Road Frames,2008-04-30 00:00:00
3,718,"HL Road Frame - Red, 44",FR-R92R-44,1,1,Red,500,375,868.6342,1431.5,44,CM,LB,2.12,1,R,H,U,14.0,6.0,2011-05-31 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000,2,Road Frames,2008-04-30 00:00:00
4,719,"HL Road Frame - Red, 48",FR-R92R-48,1,1,Red,500,375,868.6342,1431.5,48,CM,LB,2.16,1,R,H,U,14.0,6.0,2011-05-31 00:00:00,NaN,NaN,2014-02-08 10:01:36.827000000,2,Road Frames,2008-04-30 00:00:00


## Join Sales Orders and Sales Line Items Tables

Note that each "invoice" (kept track of in `orders`) can have multiple "line items" (`line_items`). This makes it a "one-to-many" merge.

* Do a 'left join" on the `orders` and `line_items` tables -- the orders table is on the left
* Because this is a "one-to-many" merge, you can optionally add a `validate="1:m"` kwarg, see the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html)
* Don't forget to check your data dictionaries if necessary!

In [54]:
print(orders.shape)
orders.head(3)

(100, 25)


,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00
1,43660,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,NaN,2011-06-07 00:00:00
2,43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,NaN,2011-06-07 00:00:00


In [56]:
print(line_items.shape)
line_items.head(15)

(65534, 9)


,LineItemID,SalesOrderID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,ModifiedDate
0,1,43659,1,776,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00
1,2,43659,3,777,2024.9940,0.0,6074.9820,4911-403C-98,2011-05-31 00:00:00
2,3,43659,1,778,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00
3,4,43659,1,771,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
4,5,43659,1,772,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
5,6,43659,2,773,2039.9940,0.0,4079.9880,4911-403C-98,2011-05-31 00:00:00
6,7,43659,1,774,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
7,8,43659,3,714,28.8404,0.0,86.5212,4911-403C-98,2011-05-31 00:00:00
8,9,43659,1,716,28.8404,0.0,28.8404,4911-403C-98,2011-05-31 00:00:00
9,10,43659,6,709,5.7000,0.0,34.2000,4911-403C-98,2011-05-31 00:00:00


In [61]:
pd.set_option('display.max_columns', 35)

full_orders = pd.merge(orders, line_items, how='inner', on='SalesOrderID', suffixes=('_o', '_li'))
print(full_orders.shape)

full_orders.head(15)

(414, 33)


,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate_o,LineItemID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,ModifiedDate_li
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,1,1,776,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,2,3,777,2024.9940,0.0,6074.9820,4911-403C-98,2011-05-31 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,3,1,778,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,4,1,771,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,5,1,772,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
5,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,6,2,773,2039.9940,0.0,4079.9880,4911-403C-98,2011-05-31 00:00:00
6,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,7,1,774,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00
7,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,8,3,714,28.8404,0.0,86.5212,4911-403C-98,2011-05-31 00:00:00
8,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,9,1,716,28.8404,0.0,28.8404,4911-403C-98,2011-05-31 00:00:00
9,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,10,6,709,5.7000,0.0,34.2000,4911-403C-98,2011-05-31 00:00:00


## Join Sales Orders, Sales Line Items, and Products Tables

* Join the `orders`, `line_items`, and `products` tables

In [73]:
pd.set_option('display.max_columns', 60)

fowpd = full_orders.merge(merged_products, how='left', on='ProductID')
print(fowpd.shape)
fowpd.head(15)

(414, 59)


,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,ModifiedDate_o,LineItemID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,ModifiedDate_li,Name_p,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ModifiedDate_p,ProductCategoryID,Name_ps,ModifiedDate_ps
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,1,1,776,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Black, 42",BK-M82B-42,1,1,Black,100,75,1898.0944,3374.99,42,CM,LB,20.77,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,2,3,777,2024.9940,0.0,6074.9820,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Black, 44",BK-M82B-44,1,1,Black,100,75,1898.0944,3374.99,44,CM,LB,21.13,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,3,1,778,2024.9940,0.0,2024.9940,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Black, 48",BK-M82B-48,1,1,Black,100,75,1898.0944,3374.99,48,CM,LB,21.42,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,4,1,771,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Silver, 38",BK-M82S-38,1,1,Silver,100,75,1912.1544,3399.99,38,CM,LB,20.35,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,5,1,772,2039.9940,0.0,2039.9940,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Silver, 42",BK-M82S-42,1,1,Silver,100,75,1912.1544,3399.99,42,CM,LB,20.77,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
5,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-07 00:00:00,6,2,773,2039.9940,0.0,4079.9880,4911-403C-98,2011-05-31 00:00:00,"Mountain-100 Silver, 44",BK-M82S-44,1,1,Silver,100,75,1912.1544,3399.99,44,CM,LB,21.13,4,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,2014-02-08 10:01:36.827000000,1,Mountain Bikes,2008-04-30 00:00:00
6,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,29825,279.0,5,985,985,5,16281.0,105041Vi84182,NaN,20565.6206,1971.5149,616.0984,23153.2339,NaN,2011-06-

* Finally, here's how to drop columns that you deem are not necessary for your analysis ([docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) on `drop`):

In [74]:
columns_to_drop = ['RevisionNumber', 'OnlineOrderFlag', 'CustomerID', 'SalesPersonID', 
                   'TerritoryID', 'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 
                   'CreditCardID', 'CreditCardApprovalCode', 'CurrencyRateID', 'Comment', 
                   'MakeFlag', 'FinishedGoodsFlag', 'SafetyStockLevel', 'ReorderPoint', 
                   'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight',
                   'DaysToManufacture', 'ProductSubcategoryID', 'ProductModelID',
                   'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'ModifiedDate_p',
                   'ProductCategoryID', 'ModifiedDate_ps', 'ModifiedDate_o', 
                   'ModifiedDate_li', 'LineItemID', 'ProductID', 'ModifiedDate_li', 
                   'ProductLine', 'Class', 'Style']

fowpd.drop(columns_to_drop, axis='columns', inplace=True)
fowpd

,SalesOrderID,OrderDate,DueDate,ShipDate,Status,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,SubTotal,TaxAmt,Freight,TotalDue,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal,CarrierTrackingNumber,Name_p,ProductNumber,Color,StandardCost,ListPrice,Name_ps
0,43659,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,SO43659,PO522145787,10-4020-000676,20565.6206,1971.5149,616.0984,23153.2339,1,2024.9940,0.0,2024.9940,4911-403C-98,"Mountain-100 Black, 42",BK-M82B-42,Black,1898.0944,3374.99,Mountain Bikes
1,43659,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,SO43659,PO522145787,10-4020-000676,20565.6206,1971.5149,616.0984,23153.2339,3,2024.9940,0.0,6074.9820,4911-403C-98,"Mountain-100 Black, 44",BK-M82B-44,Black,1898.0944,3374.99,Mountain Bikes
2,43659,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,SO43659,PO522145787,10-4020-000676,20565.6206,1971.5149,616.0984,23153.2339,1,2024.9940,0.0,2024.9940,4911-403C-98,"Mountain-100 Black, 48",BK-M82B-48,Black,1898.0944,3374.99,Mountain Bikes
3,43659,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,SO43659,PO522145787,10-4020-000676,20565.6206,1971.5149,616.0984,23153.2339,1,2039.9940,0.0,2039.9940,4911-403C-98,"Mountain-100 Silver, 38",BK-M82S-38,Silver,1912.1544,3399.99,Mountain Bikes
4,43659,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,SO43659,PO522145787,10-4020-000676,20565.6206,1971.5149,616.0984,23153.2339,1,2039.9940,0.0,2039.9940,4911-403C-98,"Mountain-100 Silver, 42",BK-M82S-42,Silver,1912.1544,3399.99,Mountain Bikes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,43754,2011-06-14 00:00:00,2011-06-26 00:00:00,2011-06-21 00:00:00,5,SO43754,NaN,10-4030-027617,3578.2700,286.2616,89.4568,3953.9884,1,3578.2700,0.0,3578.2700,NaN,"Road-150 Red, 48",BK-R93R-48,Red,2171.2942,3578.27,Road Bikes
410,43755,2011-06-14 00:00:00,2011-06-26 00:00:00,2011-06-21 00:00:00,5,SO43755,NaN,10-4030-027670,3578.2700,286.2616,89.4568,3953.9884,1,3578.2700,0.0,3578.2700,NaN,"Road-150 Red, 44",BK-R93R-44,Red,2171.2942,3578.27,Road Bikes
411,43756,2011-06-14 00:00:00,2011-06-26 00:00:00,2011-06-21 00:00:00,5,SO43756,NaN,10-4030-019941,699.0982,55.9279,17.4775,772.5036,1,699.0982,0.0,699.0982,NaN,"Road-650 Black, 58",BK-R50B-58,Black,486.7066,782.99,Road Bikes
412,43757,2011-06-14 00:00:00,2011-06-26 00:00:00,2011-06-21 00:00:00,5,SO43757,NaN,10-4030-011017,3374.9900,269.9992,84.3748,3729.3640,1,3374.9900,0.0,3374.9900,NaN,"Mountain-100 Black, 44",BK-M82B-44,Black,1898.0944,3374.99,Mountain Bikes
